**Task** **1**: **Document** **Analysis**

Identify and document common tampering patterns in:
 - Degree Certificates (PDF metadata changes, fake templates)
 - Academic Transcripts (grade edits, course list changes)
 - Professional Certifications (date extensions, level changes)


In [5]:
import pandas as pd
# Load Excel sheets
file_path = "academic_documents.xlsx"
dc_df = pd.read_excel(file_path, sheet_name="Degree_Certificates")
ac_df = pd.read_excel(file_path, sheet_name="Academic_transcripts")
pc_df = pd.read_excel(file_path, sheet_name="professionals_Certification")

flagged = []

In [7]:
# Helper function to flag tampering
def check_tampering(row, doc_type, fake_templates=[], unknown_editors=['UnknownUser', 'UnknownEditor']):
    flags = []
    if 'Last Modified' in row and 'Issue Date' in row:
        if pd.to_datetime(row['Last Modified']) > pd.to_datetime(row['Issue Date']):
            flags.append("Modified after issue")
    if 'Editor' in row and row['Editor'] in unknown_editors:
        flags.append("Unknown editor")
    if 'Template' in row and row['Template'] in fake_templates:
        flags.append("Fake template/level")
    if 'Certification Level' in row and "FAKE" in str(row['Certification Level']).upper():
        flags.append("Fake template/level")

    if flags:
        return {
            "Document ID": row.get('Document ID') or row.get('Cert ID'),
            "Name": row["Name"],
            "Flags": ", ".join(flags),
            "Document Type": doc_type
        }

In [8]:
# Process each sheet
for _, row in dc_df.iterrows():
    result = check_tampering(row, "Degree Certificate", fake_templates=["DEG-FAKE"])
    if result: flagged.append(result)

for _, row in ac_df.iterrows():
    result = check_tampering(row, "Academic Transcript", fake_templates=["TRN-FAKE"])
    if result: flagged.append(result)

for _, row in pc_df.iterrows():
    result = check_tampering(row, "Professional Certification", fake_templates=["FAKE-LVL"])
    if result: flagged.append(result)

In [9]:
# Create output DataFrame
flagged_df = pd.DataFrame(flagged)

In [10]:
# Save to Excel file
output_file = "flagged_documents.xlsx"
flagged_df.to_excel(output_file, index=False)

print(f"Tampering detection completed. Output saved to '{output_file}'")


Tampering detection completed. Output saved to 'flagged_documents.xlsx'


In [11]:
# Load the Excel file with 3 sheets
file_path = 'academic_documents.xlsx'
xls = pd.ExcelFile(file_path)

In [12]:
# Define authorized editors and valid templates/levels
authorized_editors = ['Registrar', 'UniversityAdmin', 'AdminUser', 'CertifyingOfficer']
valid_degree_templates = ['DEG-T1', 'DEG-T2']
valid_transcript_templates = ['TRN-T1', 'TRN-T2', 'TRN-T3']
valid_cert_levels = ['CERT-L1', 'CERT-L2', 'CERT-L3']

In [13]:
def analyze_sheet(df, doc_type):
    suspicious = []
    for _, row in df.iterrows():
        flags = []
        issue_date = pd.to_datetime(row['Issue Date'])
        modified_date = pd.to_datetime(row['Last Modified']) if 'Last Modified' in row else None
        editor = row['Editor']

        # Metadata mismatch
        if modified_date and modified_date > issue_date:
            flags.append("Late modification")

        if editor not in authorized_editors:
            flags.append("Unauthorized editor")

        # Type-specific tampering
        if doc_type == "Degree":
            if row['Template'] not in valid_degree_templates:
                flags.append("Fake template")
        elif doc_type == "Transcript":
            if row['Template'] not in valid_transcript_templates:
                flags.append("Fake transcript template")
        elif doc_type == "Professional":
            if row['Certification Level'] not in valid_cert_levels:
                flags.append("Invalid certification level")

        if flags:
            suspicious.append({
                'Document ID': row.get('Document ID', row.get('Cert ID')),
                'Name': row['Name'],
                'Flags': ", ".join(flags)
            })
    return suspicious

In [14]:
# Analyze each sheet
degree_df = pd.read_excel(xls, sheet_name='Degree_Certificates')
transcript_df = pd.read_excel(xls, sheet_name='Academic_transcripts')
cert_df = pd.read_excel(xls, sheet_name='professionals_Certification')

suspicious_degree = analyze_sheet(degree_df, "Degree")
suspicious_transcript = analyze_sheet(transcript_df, "Transcript")
suspicious_cert = analyze_sheet(cert_df, "Professional")


In [15]:
# Combine all suspicious entries
all_suspicious = pd.DataFrame(suspicious_degree + suspicious_transcript + suspicious_cert)

In [16]:
# Save output
all_suspicious.to_excel("suspicious_documents_report.xlsx", index=False)

print("Analysis complete. Suspicious entries saved to suspicious_documents_report.xlsx")


Analysis complete. Suspicious entries saved to suspicious_documents_report.xlsx


In [ ]:
pip install pandas fpdf PyPDF2 openpyxl


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.6 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=630afaeb28518da68c56cfff052f4e942fc21ba1f79f544e8aaf7daea32af965
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


**Task** **2**: **Prototype** **Development**

Develop a lightweight script or prototype that demonstrates:
 - PDF Metadata Analysis – Detect discrepancies like issue date mismatch, edited metadata, etc.


In [32]:
import pandas as pd
from fpdf import FPDF
from PyPDF2 import PdfReader, PdfWriter
import os

# Load Excel file
df = pd.read_excel("academic_documents.xlsx", sheet_name="Degree_Certificates")

# Output folder
os.makedirs("generated_pdfs", exist_ok=True)

# Create PDF + metadata
def create_pdf_with_metadata(row, file_path):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=f"Degree Certificate", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Name: {row['Name']}", ln=True)
    pdf.cell(200, 10, txt=f"Document ID: {row['Document ID']}", ln=True)
    pdf.output(file_path)

    # Add metadata using PyPDF2
    reader = PdfReader(file_path)
    writer = PdfWriter()
    writer.append_pages_from_reader(reader)
    metadata = {
        "/Author": row['Editor'],
        "/CreationDate": f"D:{pd.to_datetime(row['Issue Date']).strftime('%Y%m%d%H%M%S')}",
        "/ModDate": f"D:{pd.to_datetime(row['Last Modified']).strftime('%Y%m%d%H%M%S')}",
        "/Title": row.get('Template', 'DEG-T1')
    }
    writer.add_metadata(metadata)
    with open(file_path, "wb") as f:
        writer.write(f)


In [34]:
# Generate PDFs
for idx, row in df.iterrows():
    filename = f"generated_pdfs/{row['Document ID']}.pdf"
    create_pdf_with_metadata(row, filename)

print("PDFs created with metadata.")

PDFs created with metadata.


In [35]:
from PyPDF2 import PdfReader

valid_editors = ['Registrar', 'UniversityAdmin', 'AdminUser', 'CertifyingOfficer']
valid_templates = ['DEG-T1', 'DEG-T2']

flagged = []

for pdf_file in os.listdir("generated_pdfs"):
    path = os.path.join("generated_pdfs", pdf_file)
    reader = PdfReader(path)
    info = reader.metadata

    flags = []
    name = pdf_file.replace(".pdf", "")

    # Check metadata
    creation = info.get('/CreationDate', '').replace("D:", "")
    mod = info.get('/ModDate', '').replace("D:", "")
    author = info.get('/Author', '')
    template = info.get('/Title', '')

    try:
        creation_date = pd.to_datetime(creation, format='%Y%m%d%H%M%S')
        mod_date = pd.to_datetime(mod, format='%Y%m%d%H%M%S')
        if mod_date > creation_date:
            flags.append("Modified after issue")
    except:
        flags.append("Date parse error")

    if author not in valid_editors:
        flags.append("Unauthorized editor")

    if template not in valid_templates:
        flags.append("Invalid template")

    if flags:
        flagged.append({
            "Document ID": name,
            "Editor": author,
            "Template": template,
            "Flags": ", ".join(flags)
        })

In [36]:
# Save results
flagged_df = pd.DataFrame(flagged)
flagged_df.to_excel("tampered_pdf_report.xlsx", index=False)
print("Tampering report saved to 'tampered_pdf_report.xlsx'")


Tampering report saved to 'tampered_pdf_report.xlsx'
